#🏛️Indian Penal Code Chatbot: Your Legal Assistant 🗣️

In [1]:
import os
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT="basic_project_model_io"
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")  
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_TRACING_V2=True

In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
loader=PyPDFLoader(file_path="data/THE_INDIAN_PENAL_CODE.pdf")
pages=loader.load()
print(len(pages))

112


In [4]:
full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

Pages 112
Lines 5724
Words 89077
Charecters 467193


### Step-2: Split the data into Chunks

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

1957


#Step-3: Creating embeddings and Storing in Vector Stores

In [7]:
from langchain_openai import OpenAIEmbeddings
## VectorStores
from langchain_chroma import Chroma

c:\Users\WIN-10\anaconda3\Lib\site-packages\opentelemetry\proto\collector\trace\v1\trace_service_pb2_grpc.py:26: RuntimeWarning: The grpc package installed is at version 1.63.0, but the generated code in opentelemetry/proto/collector/trace/v1/trace_service_pb2_grpc.py depends on grpcio>=1.63.2. Please upgrade your grpc module to grpcio>=1.63.2 or downgrade your generated code using grpcio-tools<=1.63.0. This warning will become an error in 1.65.0, scheduled for release on June 25, 2024.
  warnings.warn(


In [8]:
embeddings = OpenAIEmbeddings()

In [10]:
IPC_rules_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="vectorstores/IPC_rules_db"
                          )

#Step-4: Conversation and Retrieval Chain

In [11]:
retriever = IPC_rules_db.as_retriever()

query = """
What is the section related to child labour.
"""
result=retriever.invoke(query,top_k=3)
result

[Document(page_content='374. Unlawful compulsory labour .—Whoever unlawfully compels any person  to labour against the \nwill of that person, shall be punished with imprisonment of either description for a term which may \nextend to one ye ar, or with fine, or with both.  \n4[Sexual offences', metadata={'page': 83, 'source': 'data/THE_INDIAN_PENAL_CODE.pdf'}),
 Document(page_content='374. Unlawful compulsory labour .—Whoever unlawfully compels any person  to labour against the \nwill of that person, shall be punished with imprisonment of either description for a term which may \nextend to one ye ar, or with fine, or with both.  \n4[Sexual offences', metadata={'page': 83, 'source': 'data/THE_INDIAN_PENAL_CODE.pdf'}),
 Document(page_content='rupees, or with both.  \nExplanation .—The last section is subject to the s ame Explanation  as section 352.  \nOf Kidnapping, Abduc tion, Slavery and Forced Labour  \n359. Kidnapping .—Kidnapping is of two kinds: kidnapping from 1[India], and kidna 

In [12]:
#result[0].page_content
len(result)

4

In [13]:
for i in range(len(result)):
     print(result[i].page_content)

374. Unlawful compulsory labour .—Whoever unlawfully compels any person  to labour against the 
will of that person, shall be punished with imprisonment of either description for a term which may 
extend to one ye ar, or with fine, or with both.  
4[Sexual offences
374. Unlawful compulsory labour .—Whoever unlawfully compels any person  to labour against the 
will of that person, shall be punished with imprisonment of either description for a term which may 
extend to one ye ar, or with fine, or with both.  
4[Sexual offences
rupees, or with both.  
Explanation .—The last section is subject to the s ame Explanation  as section 352.  
Of Kidnapping, Abduc tion, Slavery and Forced Labour  
359. Kidnapping .—Kidnapping is of two kinds: kidnapping from 1[India], and kidna pping from 
lawful guardianship.
rupees, or with both.  
Explanation .—The last section is subject to the s ame Explanation  as section 352.  
Of Kidnapping, Abduc tion, Slavery and Forced Labour  
359. Kidnapping .—Kidna

In [18]:

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI

In [19]:
llm=ChatOpenAI(temperature=0)

In [27]:
from langchain.chains import ConversationalRetrievalChain
memory = ConversationBufferMemory()

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversational_RAG = ConversationalRetrievalChain.from_llm(llm=llm,
                                                     retriever=IPC_rules_db.as_retriever(),
                                                     memory=memory,
                                                     #verbose=True
                                                     )




In [28]:


user_input=input("Your message:")
while user_input!="quit":
    print("AI message ==>", conversational_RAG.invoke(input=user_input))
    user_input=input("Your message: ")
while user_input =="quit":
    memory.clear()
    break